In [3]:
import yfinance as yf
from datetime import date, datetime
import pandas as pd 
import numpy as np

In [11]:
start_date = date(2000, 2, 1)
end_date = date(2021, 3, 12)

tickerSymbol = '^GSPC'
tickerData = yf.Ticker(tickerSymbol)

df = tickerData.history(period='1d', start=start_date, end=end_date).dropna()
df.drop(['Stock Splits', 'Dividends'], axis='columns', inplace=True)
df = pd.DataFrame(data = df)
df.index = pd.to_datetime(df.index)
# df["Date"] = df.index

df_copy = tickerData.history(period='1d', start=start_date, end=end_date).dropna()

df_copy.head(3)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,0,0
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0,0
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,0,0


# Simple Moving Average

In [5]:
def SMA(data,n):
    
    #додаємо NaN на початку
	sma_list=[np.nan for k in range(n-1)]
    
	for i in range(len(data)-n+1):
		sum_n = 0
		for j in range(n):
			sum_n = sum_n + data[i+j]
		sma=sum_n/n
		sma_list.append(round(sma, 2))
        
	return sma_list

# Linear Weighted Moving Average

In [6]:
def LWMA(data,n):
    
	lma_list=[np.nan for k in range(n-1)]

	for i in range(len(data)-n+1):
		sum_n, sum_j = 0, 0
		for j in range(n):
			sum_n = sum_n + (j+1)*data[i+j]
			sum_j = sum_j + (j+1)
		lma=sum_n/sum_j
		lma_list.append(lma)

	return lma_list

# Exponential Moving Average

In [7]:
def EMA(data, n, a=0):
	ema_list=[data[0]]
	if a==0:
		a=2/(n+1)

	for i in range (1,len(data)):
		ema = 0
		ema = (1-a)*ema_list[i-1]+a*data[i]
		ema_list.append(ema)

	return ema_list

# SO

In [8]:
def SO(data_close, data_high, data_low, n):
	K, D = [np.nan for k in range(n-1)], [np.nan for k in range(n-1)]
	highest_high, lowest_low = [np.nan for k in range(n-1)], [np.nan for k in range(n-1)]

	for i in range(len(data_close)-n+1):
		list_n_high, list_n_low = [], []
		for j in range(0, n):
			list_n_high.append(data_high[i+j])
			list_n_low.append(data_low[i+j])
		highest_high.append(max(list_n_high))
		lowest_low.append(min(list_n_low))

	for i in range(n-1, len(highest_high)):
		K.append(100*(data_close[i]-lowest_low[i])/(highest_high[i]-lowest_low[i]))
	D = SMA(K, n)

	return K, D

# ERI

In [21]:
def ERI(data_close, data_high, data_low, n):
    MA = EMA(data_close, n)
    print(MA)
    BuP, BeP = [], []
    for i in range(len(data_close)):
        BuP.append(data_high[i] - MA[i])
        BeP.append(data_low[i] - MA[i])
    return BuP, BeP# Elder-Ray Index

# CCI

In [10]:
def CCI(data_close, data_high, data_low, n):
	p_typical = []

	for i in range(len(data_close)):
		p_typical.append((data_close[i]+data_low[i]+data_high[i])/3)
	sma = SMA(p_typical,n)

	MAD = [np.nan for k in range(n-1)]
	for j in range(n-1, len(data_close)):
		mad = 0
		for s in range(n):
			mad = mad +  abs(p_typical[j-s]-sma[j])
		MAD.append(mad/n)

	CCI = [np.nan for n in range(n-1)]
	for t in range(n-1, len(data_close)):
		cci = (p_typical[t]-sma[t])/(0.015*MAD[t])
		CCI.append(cci)

	return CCI

In [18]:
df_copy["SO 5 K"] = SO(df_copy.Close, df_copy.High, df_copy.Low, 5)[0]
df_copy["SO 5 D"] = SO(df_copy.Close, df_copy.High, df_copy.Low, 5)[1]
df_copy["ERI 5 BuP"] = ERI(df_copy.Close, df_copy.High, df_copy.Low, 5)[0]
df_copy["ERI 5 BeP"] = ERI(df_copy.Close, df_copy.High, df_copy.Low, 5)[1]
df_copy["CCI"] = CCI(df_copy.Close, df_copy.High, df_copy.Low, 5)
df_copy.tail()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ERI 5 BuP,ERI 5 BeP,CCI,SO 5 K,SO 5 D
Date,,,,,,,,,,,,
2021-03-05,3793.58,3851.69,3730.19,3841.94,6842570000,0,0,20.628409,-100.871591,-55.736326,62.042268,49.34
2021-03-08,3844.39,3881.06,3819.25,3821.35,5852240000,0,0,53.235606,-8.574394,26.023736,53.536898,43.90
2021-03-09,3851.93,3903.76,3851.93,3875.44,5496340000,0,0,60.063737,8.233737,115.542165,84.303292,49.15
2021-03-10,3891.99,3917.35,3885.73,3898.81,5827250000,0,0,55.282492,23.662492,103.853179,90.443792,62.79
2021-03-11,3915.54,3960.27,3915.54,3939.34,5300010000,0,0,72.444994,27.714994,112.119552,90.903164,76.25


In [19]:
df_copy.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits,ERI 5 BuP,ERI 5 BeP,CCI,SO 5 K,SO 5 D
Date,,,,,,,,,,,,
2000-01-31,1360.16,1394.48,1350.14,1394.46,993800000,0,0,0.020000,-44.320000,NaN,NaN,NaN
2000-02-01,1394.46,1412.49,1384.79,1409.28,981000000,0,0,13.090000,-14.610000,NaN,NaN,NaN
2000-02-02,1409.28,1420.61,1403.49,1409.12,1038600000,0,0,17.970000,0.850000,NaN,NaN,NaN
2000-02-03,1409.12,1425.78,1398.52,1424.97,1146500000,0,0,15.696667,-11.563333,NaN,NaN,NaN
2000-02-04,1424.97,1435.91,1420.63,1424.37,1045100000,0,0,21.064444,5.784444,100.535851,86.545412,NaN


In [20]:
df_copy.to_excel('d:\Курсова\Курсова 2021\check 3.xlsx')